# Discovery

Required packages: vfb-connect and python-catmaid (pymaid & navis)

In [1]:
!pip install -r ../../../requirements.txt --quiet
!pip install ../../../ --quiet
!pip install python-catmaid --quiet

### How to find neurons across datasets

VirtualFlyBrain integrates images and connectomics profiles of neurons from many sources.  It classifies and records their properties using a standard, queryable classification ([The Drosophila Anatomy Ontology](https://www.ebi.ac.uk/ols/ontologies/fbbt)). This standardises the names of neuron types across sources, so you don't need to worry about differences in nomenclature uses and supports queries for neurons by their classification. 

In [ ]:
# Import libs and initialise API objects
from vfb_connect import vfb
import pandas as pd
import os


import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Configure VFB for CI/notebook execution performance
vfb._load_limit = int(os.getenv('VFB_LOAD_LIMIT', 50))  # Default to 50 for discovery examples
if os.getenv('VFB_CACHE_ENABLED', 'false').lower() == 'true':
    vfb._use_cache = True

print(f"VFB load limit set to: {vfb._load_limit}")

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


### Finds neurons by type (classification) across datasets

We can use the `vfb.get_instances` method in combination with the name of a neuron type on VFB to find individual neurons from multiple sources.

Use the search tool on [VFB](http://virtualflybrain.org) to find neuron types by name or synonym:

<img src="https://user-images.githubusercontent.com/112839/109564687-535c9380-7ad9-11eb-80e9-5a5bc21cd915.png" width=30% height=30%>
<img src="https://user-images.githubusercontent.com/112839/109565128-eac1e680-7ad9-11eb-9649-1ec55a298a1b.png" width=30% height=30%>

Use either the full name or the Symbol to query for neurons:

In [ ]:
DA3adPN = vfb.get_instances("adult Drosulfakinin neuron", return_dataframe=True)
DA3adPN


,label,symbol,id,tags,data_source,accession,parents_label,parents_id,xrefs,templates,dataset,license
0,DSKMP1A_R (FlyEM-HB:1135837629),DSKMP1A_R,VFB_jrchjti2,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1135837629],"[adult octopaminergic neuron, adult Drosulfaki...","[FBbt_00058203, FBbt_00051433]","[neuprint_JRC_Hemibrain_1point2point1:1135837629, ne...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
1,DSKMP1A(PVM02)_L (FlyEM-HB:1260833150),DSKMP1A(PVM02)_L,VFB_jrchjti3,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1260833150],"[adult Drosulfakinin MP1a neuron, adult octopa...","[FBbt_00051433, FBbt_00058203]","[neuronbridge:1260833150, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
2,DSKMP1B_R (FlyEM-HB:1352077058),DSKMP1B_R,VFB_jrchjti4,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1352077058],"[adult octopaminergic neuron, adult Drosulfaki...","[FBbt_00058203, FBbt_00051434]","[neuprint_JRC_Hemibrain_1point2point1:1352077058, ne...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
3,DSKMP1B(PVM02)_L (FlyEM-HB:1011184205),DSKMP1B(PVM02)_L,VFB_jrchjti5,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1011184205],"[adult Drosulfakinin MP1b neuron, adult octopa...","[FBbt_00051434, FBbt_00058203]","[neuprint_JRC_Hemibrain_1point2point1:1011184205, ne...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
4,DSKMP3_R (FlyEM-HB:328559607),DSKMP3_R,VFB_jrchjti7,"[Entity, Adult, Anatomy, Cell, Cholinergic, In...",[neuprint_JRC_Hemibrain_1point2point1],[328559607],"[adult cholinergic neuron, adult Drosulfakinin...","[FBbt_00058205, FBbt_00051432]","[neuronbridge:328559607, neuprint_JRC_Hemibrai...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
5,DSKMP3_R (FlyEM-HB:327937328),DSKMP3_R,VFB_jrchjti6,"[Entity, Adult, Anatomy, Cell, Cholinergic, In...",[neuprint_JRC_Hemibrain_1point2point1],[327937328],"[adult cholinergic neuron, adult Drosulfakinin...","[FBbt_00058205, FBbt_00051432]","[neuprint_JRC_Hemibrain_1point2point1:327937328, neu...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...


### Find neurons by location

We can use the same method to search for neurons by location, using simple queries.

In [ ]:
# Find neurons by location. The following query works across multiple data sources and both sides of the brain.  
# Results may be incomplete & may include minor overlap inferred from low synapse counts

neurons_in_DA3 = vfb.get_instances("'neuron' that 'overlaps' some 'antennal lobe glomerulus DA3'", return_dataframe=True)
neurons_in_DA3[0:5]


,label,symbol,id,tags,data_source,accession,parents_label,parents_id,xrefs,templates,dataset,license
0,v2LN35_R (FlyEM-HB:1733056086),v2LN35_R,VFB_jrchk8ew,"[Entity, Adult, Anatomy, Cell, Cholinergic, In...",[neuprint_JRC_Hemibrain_1point2point1],[1733056086],[cholinergic local interneuron of the adult an...,"[FBbt_00007403, FBbt_20003766]","[neuronbridge:1733056086, neuprint_JRC_Hemibra...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
1,lLN2T_a(Tortuous)_R (FlyEM-HB:5813032595),lLN2T_a(Tortuous)_R,VFB_jrchk8bd,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[5813032595],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuronbridge:5813032595, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
2,lLN2T_d(Tortuous)_R (FlyEM-HB:1667251683),lLN2T_d(Tortuous)_R,VFB_jrchk8bp,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1667251683],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuronbridge:1667251683, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
3,lLN2P_b(Patchy)_R (FlyEM-HB:1946178096),lLN2P_b(Patchy)_R,VFB_jrchk8au,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1946178096],[adult local interneuron of the lateral ALl1 n...,"[FBbt_00007394, FBbt_00007394, FBbt_00007394, ...","[neuprint_JRC_Hemibrain_1point2point1:1946178096, ne...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
4,lLN2T_a(Tortuous)_R (FlyEM-HB:1762354941),lLN2T_a(Tortuous)_R,VFB_jrchk8bg,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1762354941],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuprint_JRC_Hemibrain_1point2point1:1762354941, ne...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...


In [ ]:
# Find local interneurons (intrinsic neurons) of the AL, overlapping DA3:

local_in_DA3 = vfb.get_instances("'local interneuron of adult antennal lobe' that 'overlaps' some 'antennal lobe glomerulus DA3'", return_dataframe=True)
local_in_DA3

,label,symbol,id,tags,data_source,accession,parents_label,parents_id,xrefs,templates,dataset,license
0,v2LN35_R (FlyEM-HB:1733056086),v2LN35_R,VFB_jrchk8ew,"[Entity, Adult, Anatomy, Cell, Cholinergic, In...",[neuprint_JRC_Hemibrain_1point2point1],[1733056086],[cholinergic local interneuron of the adult an...,"[FBbt_00007403, FBbt_20003766]","[neuronbridge:1733056086, neuprint_JRC_Hemibra...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
1,lLN2T_a(Tortuous)_R (FlyEM-HB:5813032595),lLN2T_a(Tortuous)_R,VFB_jrchk8bd,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[5813032595],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuronbridge:5813032595, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
2,lLN2T_d(Tortuous)_R (FlyEM-HB:1667251683),lLN2T_d(Tortuous)_R,VFB_jrchk8bp,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1667251683],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuronbridge:1667251683, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
3,lLN2P_b(Patchy)_R (FlyEM-HB:1946178096),lLN2P_b(Patchy)_R,VFB_jrchk8au,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1946178096],[adult local interneuron of the lateral ALl1 n...,"[FBbt_00007394, FBbt_00007394, FBbt_00007394, ...","[neuprint_JRC_Hemibrain_1point2point1:1946178096, ne...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
4,lLN2T_a(Tortuous)_R (FlyEM-HB:1762354941),lLN2T_a(Tortuous)_R,VFB_jrchk8bg,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1762354941],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuprint_JRC_Hemibrain_1point2point1:1762354941, ne...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
5,lLN1_c_R (FlyEM-HB:1547454812),lLN1_c_R,VFB_jrchk8ai,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1547454812],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuprint_JRC_Hemibrain_1point2point1:1547454812, ne...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
6,lLN1_b_R (FlyEM-HB:5813054622),lLN1_b_R,VFB_jrchk8a8,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[5813054622],"[adult antennal lobe local neuron lLN1, adult ...","[FBbt_00007395, FBbt_00049526, FBbt_00049526, ...","[neuronbridge:5813054622, neuprint_JRC_Hemibra...","[JRC_FlyEM_Hemibrain, JRC2018Unisex]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
7,lLN2T_a(Tortuous)_R (FlyEM-HB:5813056598),lLN2T_a(Tortuous)_R,VFB_jrchk8bi,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[5813056598],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuprint_JRC_Hemibrain_1point2point1:5813056598, ne...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...
8,lLN2T_e(Tortuous)_R (FlyEM-HB:1640922516),lLN2T_e(Tortuous)_R,VFB_jrchk8bs,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[neuprint_JRC_Hemibrain_1point2point1],[1640922516],"[adult serotonergic neuron, adult serotonergic...","[FBbt_00049526, FBbt_00049526, FBbt_00049526, ...","[neuprint_JRC_Hemibrain_1point2point1:1640922516, ne...","[JRC_FlyEM_Hemibrain, JR

In [ ]:
# Find neurons by dataset/paper - on CATMAID

bates = pymaid.find_neurons(annotations='Paper: Bates and Schlegel et al 2020')
bates

INFO  : Found 584 neurons matching the search parameters (pymaid)


,type,name,skeleton_id,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,CatmaidNeuron,Uniglomerular mALT DA1 lPN 57316 2863105 ML,2863104,NA,NA,NA,NA,NA,NA,1 nanometer
1,CatmaidNeuron,Uniglomerular mALT DA3 adPN 57350 HG,57349,NA,NA,NA,NA,NA,NA,1 nanometer
...,...,...,...,...,...,...,...,...,...,...
582,CatmaidNeuron,Multiglomerular mlALT vPN VM7d+VM5d+DC4+6 LTS ...,4624378,NA,NA,NA,NA,NA,NA,1 nanometer
583,CatmaidNeuron,Uniglomerular mALT DL2d adPN 57342 ML,57341,NA,NA,NA,NA,NA,NA,1 nanometer


In [ ]:
# Inspect what datasets are available on VFB

ds = vfb.get_datasets()
ds
ds.sort_values(by=['id'])

,label,symbol,id,tags,description,miniref,FlyBase,PMID,DOI
76,EM L1 Andrade et al. 2019,,Andrade2019,"[Entity, DataSet, Individual, has_image, L1EM]",[L1 EM reconstructed neurons from Andrade et a...,"[Andrade et al., 2019, Curr. Biol. 29(3): 412-...",[FBrf0241389],[30661802],[10.1016/j.cub.2018.12.012]
30,MBONs and split-GAL4 lines that target them (A...,,Aso2014,"[Entity, DataSet, Individual, has_image]",[],"[Aso et al., 2014, eLife 3: e04577]",[FBrf0227179],[25535793],[10.7554/eLife.04577]
72,split-GAL4 lines for dopaminergic neurons (Aso...,,AsoRubin2016,"[Entity, DataSet, Individual, has_image]",[For comparison of the properties of memories ...,"[Aso and Rubin, 2016, eLife 5: e16135]",[FBrf0233230],[27441388],[10.7554/eLife.16135]
165,EM FAFB Baltruschat et al 2021,,Baltruschat2021,"[Entity, DataSet, Individual, has_image, FAFB]",[FAFB EM reconstructed neurons from Baltruscha...,"[Baltruschat et al., 2021, Cell Rep. 34(11): 1...",[FBrf0248487],[33730583],[10.1016/j.celrep.2021.108871]
29,"EM L1 Barnes et al., 2022",,Barnes2022,"[Entity, DataSet, Individual, L1EM]",[L1 EM reconstructed neurons from Barnes et al...,"[Barnes et al., 2022, PLoS ONE 17(4): e0266064]",[FBrf0253082],[35377898],[10.1371/journal.pone.0266064]
...,...,...,...,...,...,...,...,...,...
95,"EM L1 Zarin, Mark et al. 2019",,Zarin2019,"[Entity, DataSet, Individual, has_image, L1EM]","[L1 EM reconstructed neurons from Zarin, Mark ...","[Zarin, Mark et al., 2019]",[],[],[10.1101/617977]
93,"EM FAFB Zhao et al., 2023",,Zhao2023,"[Entity, DataSet, Individual, has_image, FAFB]",[FAFB EM reconstructed neurons from Zhao et al...,"[Zhao et al., 2023]",[],[],[10.1101/2022.12.14.520178]
47,EM FAFB Zheng et al 2018,,Zheng2018,"[Entity, DataSet, Individual, has_image, FAFB]",[FAFB EM reconstructed neurons from Zheng et a...,"[Zheng et al., 2018, Cell 174(3): 730--743.e22]",[FBrf0239557],[30033368],[10.1016/j.cell.2018.06.019]
48,EM FAFB Zheng et al 2020 (published 2022),,Zheng2020,"[Entity, DataSet, Individual, has_image, FAFB]",[FAFB EM reconstructed neurons from Zheng et a...,"[Zheng et al., 2022, Curr. Biol. 32(15): 3334-...",[FBrf0254218],[35797998],[10.1016/j.cub.2022.06.031]


In [ ]:
sayin_tab = vfb.get_instances_by_dataset('Sayin2019')
sayin_tab

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,xrefs,templates,dataset,license
0,OA-VPM3 (FAFB:1329078),,VFB_001001dr,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[octopaminergic VPM3 neuron],[FBbt_00110151],[catmaid_fafb],[1329078],[catmaid_fafb:1329078],[JRC2018Unisex],[Sayin2019],[https://creativecommons.org/licenses/by-sa/4....
1,OA-VPM4 (FAFB:1191261),,VFB_001001dq,"[Entity, Adult, Anatomy, Cell, Individual, Ner...",[octopaminergic VPM4 neuron],[FBbt_00110152],[catmaid_fafb],[1191261],[catmaid_fafb:1191261],[JRC2018Unisex],[Sayin2019],[https://creativecommons.org/licenses/by-sa/4....


In [ ]:
vfb.get_connected_neurons_by_type(upstream_type='LNd', downstream_type='adult descending neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjzxx,LNd_R (FlyEM-HB:5813021192),25,VFB_jrchjthm,DNp27_R (FlyEM-HB:1072063538),LNd neuron,descending neuron of the posterior brain DNp27...,None,None,None,None


In [ ]:
# Intra pacemaker neuron neuron connections

vfb.get_connected_neurons_by_type(upstream_type='pacemaker neuron', downstream_type='pacemaker neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchk8e0,5th s-LNv (FlyEM-HB:511051477),25,VFB_jrchjzxx,LNd_R (FlyEM-HB:5813021192),adult cholinergic neuron|adult s-LNv neuron,LNd neuron,None,None,None,None
1,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),21,VFB_jrchjzxx,LNd_R (FlyEM-HB:5813021192),adult cholinergic neuron|LNd neuron,LNd neuron,None,None,None,None
2,VFB_jrchjtf3,DN1pA_R (FlyEM-HB:387166379),30,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),ventro-contralateral-projecting DN1p neuron|ad...,adult cholinergic neuron|LNd neuron,None,None,None,None
3,VFB_jrchjtex,DN1a_R (FlyEM-HB:264083994),79,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1a neuron,adult cholinergic neuron|LNd neuron,None,None,None,None
4,VFB_jrchjtf1,DN1pA_R (FlyEM-HB:325529237),30,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),adult glutamatergic neuron|ventro-contralatera...,adult cholinergic neuron|LNd neuron,None,None,None,None
5,VFB_jrchjtey,DN1a_R (FlyEM-HB:5813022274),75,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),DN1a neuron,adult cholinergic neuron|LNd neuron,None,None,None,None
6,VFB_jrchk8e0,5th s-LNv (FlyEM-HB:511051477),40,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),adult cholinergic neuron|adult s-LNv neuron,adult cholinergic neuron|LNd neuron,None,None,None,None
7,VFB_jrchjtf2,DN1pA_R (FlyEM-HB:387944118),34,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),adult glutamatergic neuron|ventro-contralatera...,adult cholinergic neuron|LNd neuron,None,None,None,None
8,VFB_jrchjtf0,DN1pA_R (FlyEM-HB:324846570),37,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),ventro-contralateral-projecting DN1p neuron|ad...,adult cholinergic neuron|LNd neuron,None,None,None,None
9,VFB_jrchjzxy,LNd_R (FlyEM-HB:5813069648),29,VFB_jrchjzxw,LNd_R (FlyEM-HB:5813056917),adult cholinergic neuron|LNd neuron,adult cholinergic neuron|LNd neuron,None,None,None,None


In [ ]:

vfb.get_connected_neurons_by_type(upstream_type='adult neuron', downstream_type='adult Drosulfakinin neuron', weight=20)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_jrchjrmu,AVLP053_R (FlyEM-HB:5813021533),124,VFB_jrchjti2,DSKMP1A_R (FlyEM-HB:1135837629),adult anterior ventrolateral protocerebrum neu...,adult octopaminergic neuron|adult Drosulfakini...,None,None,None,None
1,VFB_jrchjs4y,AVLP340_R (FlyEM-HB:1260949492),29,VFB_jrchjti2,DSKMP1A_R (FlyEM-HB:1135837629),adult anterior ventrolateral protocerebrum neu...,adult octopaminergic neuron|adult Drosulfakini...,None,None,None,None
2,VFB_jrchk5in,SLP130_R (FlyEM-HB:456847100),94,VFB_jrchjti2,DSKMP1A_R (FlyEM-HB:1135837629),adult superior lateral protocerebrum neuron 130,adult octopaminergic neuron|adult Drosulfakini...,None,None,None,None
3,VFB_jrchjsi7,AVLP580_L (FlyEM-HB:1097642017),91,VFB_jrchjti2,DSKMP1A_R (FlyEM-HB:1135837629),adult glutamatergic neuron|adult anterior vent...,adult octopaminergic neuron|adult Drosulfakini...,None,None,None,None
4,VFB_jrchjsi9,AVLP580_L (FlyEM-HB:5812991151),64,VFB_jrchjti2,DSKMP1A_R (FlyEM-HB:1135837629),adult anterior ventrolateral protocerebrum neu...,adult octopaminergic neuron|adult Drosulfakini...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
56,VFB_jrchk5zh,SLP384_R (FlyEM-HB:482702027),20,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult superior lateral protocerebrum neuron 384,adult cholinergic neuron|adult Drosulfakinin M...,None,None,None,None
57,VFB_jrchk5ch,SLP031_R (FlyEM-HB:5813026615),59,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult superior lateral protocerebrum neuron 031,adult cholinergic neuron|adult Drosulfakinin M...,None,None,None,None
58,VFB_jrchk0fd,MBON23(a2sp)_R (FlyEM-HB:423382015),21,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),mushroom body output neuron 23|adult cholinerg...,adult cholinergic neuron|adult Drosulfakinin M...,None,None,None,None
59,VFB_jrchjz3e,LHAV6h1_R (FlyEM-HB:672960748),27,VFB_jrchjti6,DSKMP3_R (FlyEM-HB:327937328),adult glutamatergic neuron|adult lateral horn ...,adult cholinergic neuron|adult Drosulfakinin M...,None,None,None,None


### Transcriptomics Queries

# Finding single cell transcriptomics data for a given cell type

In [ ]:

profile = vfb.get_transcriptomic_profile(cell_type = 'adult olfactory receptor neuron')
print(profile.head())

                                  cell_type   cell_type_id       sample_sex  \
0  adult antennal olfactory receptor neuron  FBbt_00007352  female organism   
1  adult antennal olfactory receptor neuron  FBbt_00007352    male organism   
2  adult antennal olfactory receptor neuron  FBbt_00007352             None   
3  adult antennal olfactory receptor neuron  FBbt_00007352  female organism   
4  adult antennal olfactory receptor neuron  FBbt_00007352    male organism   

                                       sample_tissue  \
0  [antenna, adult corpus cardiacum, antenna, adu...   
1  [wing, haltere, adult, adult pars intercerebra...   
2  [antenna, proboscis, male reproductive system,...   
3  [antenna, adult corpus cardiacum, antenna, adu...   
4  [wing, haltere, adult, adult pars intercerebra...   

                                            ref     gene      gene_id  \
0  Li et al., 2022, Science 375(6584): eabk2432  14-3-3ε  FBgn0020238   
1  Li et al., 2022, Science 375(6584): eab

These results can be filtered by gene function - first find available gene functions

In [ ]:
functions = vfb.get_gene_function_filters()
print(functions)

['Acetylcholine_receptor', 'Calcium_binding', 'Carbohydrate_binding', 'Chaperone', 'Cytoskeleton_binding', 'DNA_binding', 'Dopamine_receptor', 'Enzyme', 'GABA_receptor', 'GPCR', 'Glutamate_receptor', 'Gustatory_receptor', 'Histamine_receptor', 'Hormone', 'Ion_channel', 'Ligand', 'Lipid_binding', 'Mechanosensory_ion_channel', 'Metal_ion_binding', 'Neuropeptide', 'Neurotransmitter_receptor', 'Octopamine_receptor', 'Odorant_binding', 'Olfactory_receptor', 'Peptide_or_protein_hormone_receptor', 'Photoreceptor', 'RNA_binding', 'Receptor', 'Regulator', 'Serotonin_receptor', 'Small_molecule_binding', 'Structural_molecule', 'Thermosensory_ion_channel', 'Transcription_factor', 'Transporter', 'Tyramine_receptor']


Then run transcriptomics query speicfying a gene function to filter on

In [ ]:
filtered_profile = vfb.get_transcriptomic_profile(cell_type = 'adult olfactory receptor neuron', gene_type='Olfactory_receptor')
print(filtered_profile.head())

                                  cell_type   cell_type_id       sample_sex  \
0  adult antennal olfactory receptor neuron  FBbt_00007352  female organism   
1  adult antennal olfactory receptor neuron  FBbt_00007352    male organism   
2  adult antennal olfactory receptor neuron  FBbt_00007352             None   
3           adult olfactory receptor neuron  FBbt_00007386  female organism   
4           adult olfactory receptor neuron  FBbt_00007386    male organism   

                                       sample_tissue  \
0  [antenna, adult corpus cardiacum, antenna, adu...   
1  [wing, haltere, adult, adult pars intercerebra...   
2  [antenna, proboscis, male reproductive system,...   
3                                       [adult head]   
4                                       [adult head]   

                                            ref     gene      gene_id  \
0  Li et al., 2022, Science 375(6584): eabk2432     Orco  FBgn0037324   
1  Li et al., 2022, Science 375(6584): eab